In [66]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 


In [67]:
dfinfluence = pd.read_csv('input/influence_data.csv', encoding='unicode_escape')

In [68]:
# print(dfinfluence)

## 建立artists,记录全部艺术家

In [79]:
artists = pd.DataFrame(columns= ['name', 'id', 'decade', 'genre'])
name2art = {}
id2art = {}
# for index, row in df.iterrows():
for index, row in dfinfluence.iterrows():
    id = row['influencer_id']
    name = row['influencer_name']
    # if (id not in id2art) & (name in name2art):
    #     print('name:', name)
    #     # print(name2art)
    #     artist = artists.loc[name2art[name]]
    #     print(id, ' ', artist['id'])
    # if (id in id2art) & (name not in name2art):
    if id in id2art:
        # print('id:', id)
        artist = artists.loc[id2art[id]]
        if artist['name'] != name:
            print("index=", index, "id=", id, ", names= ", artist['name'], ' ', name)
        else:
            continue
    else:
        artists.loc[artists.shape[0]] = [name, id, row['influencer_active_start'], row['influencer_main_genre']]
    # artists.append([name, id], ignore_index=True)
    # print(artists)
    # name2art[name] = artists.shape[0] - 1
        id2art[id] = artists.shape[0] - 1

for index, row in dfinfluence.iterrows():
    id = row['follower_id']
    name = row['follower_name']
    if id in id2art:
        artist = artists.loc[id2art[id]]
        if artist['name'] != name:
            print("index=", index, "id= ", id, "names= ", artist['name'], ' ', name)
        else:
            continue
    else:
        artists.loc[artists.shape[0]] = [name, id, row['follower_active_start'], row['follower_main_genre']]
        id2art[id] = artists.shape[0] - 1


index= 6644 id= 893383 , names=  The Muffs   The Kills
index= 29381 id= 303506 , names=  Usher   Day26
index= 37697 id= 957340 , names=  Helloween   Nightwish


## 修改有错误的数据

In [80]:
dfinfluence_ = dfinfluence.copy()
# print(dfinfluence_.loc[6644])
dfinfluence_.at[6644, 'influencer_name'] = 'The Muffs'
# print(dfinfluence_.loc[29381])
dfinfluence_.at[29381, 'influencer_name'] = 'Usher'
# print(dfinfluence_.loc[37697])
dfinfluence_.at[37697, 'influencer_name'] = 'Helloween'
print(dfinfluence_.loc[37697])
print(dfinfluence.loc[37687])

influencer_id                 957340
influencer_name            Helloween
influencer_main_genre       Pop/Rock
influencer_active_start         1980
follower_id                   868628
follower_name              Nightwish
follower_main_genre         Pop/Rock
follower_active_start           1990
Name: 37697, dtype: object
influencer_id                     771438
influencer_name            Black Sabbath
influencer_main_genre           Pop/Rock
influencer_active_start             1960
follower_id                       868616
follower_name                 Nickelback
follower_main_genre             Pop/Rock
follower_active_start               1990
Name: 37687, dtype: object


## 看看一共有多少个艺术家（主要的核查点）

In [81]:
# print(artists)
print(artists.shape)
print(len(id2art))

## 核查可以从下面这些开始
arr1 = dfinfluence['influencer_id'].unique()
# print(len(arr1))
# print(arr2)
# print(arr1 + arr2)
arr2 = dfinfluence['follower_id'].unique()
# list(set(a) | set(b))
arr = list(set(arr1) | set(arr2))
print(len(arr))
arr1 = dfinfluence['influencer_name'].unique()
arr2 = dfinfluence['follower_name'].unique()
arr = list(set(arr1) | set(arr2))
print(len(arr))


(5603, 4)
5603
5603
5568


# 补充artist

In [82]:
# df = pd.DataFrame({'A': [12, 23], 'B': [['a', 'b'], ['c', 'd']]})
# print(df)


## 补充图
artists 中的`towards`和`from`中的值的类型$\in$`artists.index`

In [83]:
# print(id2art)
artists['towards'] = pd.Series([[]] * artists.shape[0], index = artists.index)
artists['from'] = pd.Series([[]] * artists.shape[0], index = artists.index)
# print(pd.Series([[]] * artists.shape[0]))
# print(artists.head())
# artists.at[1, 'from'] = [1]
# print(artists.head())

for index, row in dfinfluence_.iterrows():
    iart_inf = id2art[row['influencer_id']]
    iart_fol = id2art[row['follower_id']]
    artists.at[iart_inf, 'towards'] = artists.at[iart_inf, 'towards'] + [iart_fol]
    artists.at[iart_fol, 'from'] = artists.at[iart_fol, 'from'] + [iart_inf]

print(artists.head())

            name      id decade       genre  \
0  The Exploited  759491   1980    Pop/Rock   
1         Tricky   25462   1990  Electronic   
2      Bob Dylan   66915   1960    Pop/Rock   
3  Leonard Cohen   71209   1950    Pop/Rock   
4   The Gun Club   91438   1980    Pop/Rock   

                                             towards  \
0  [3774, 3919, 1382, 1383, 1544, 211, 4530, 1364...   
1  [489, 1626, 3130, 3545, 4169, 2966, 4367, 1539...   
2  [489, 678, 42, 2692, 2816, 1888, 648, 276, 462...   
3  [489, 3223, 840, 1889, 3949, 3682, 2657, 6, 24...   
4  [489, 947, 3177, 714, 1831, 2954, 17, 1750, 5161]   

                                                from  
0  [680, 1171, 1359, 1315, 1838, 1599, 361, 1675,...  
1  [777, 1005, 1006, 478, 1007, 1008, 228, 195, 1...  
2  [1697, 1375, 1446, 1698, 1699, 233, 1700, 1447...  
3  [2, 1486, 1757, 424, 1134, 1054, 480, 425, 673...  
4  [679, 352, 152, 155, 1839, 868, 1568, 852, 194...  


In [84]:
# print(artists.iloc[489])

## 补充影响人数
`influenced_numbers` 为一个artist的影响人数

In [95]:
artists['influenced_numbers'] = pd.Series([0] * artists.shape[0])
for index, artist in artists.iterrows():
    # 被多少人影响
    arr = artist['from']
    influencer_num = len(arr)
    # print(influencer_num)
    if influencer_num == 0:
        continue 
    for influencer in arr:
        # print(art_inf)
        # print(arr)
        artists.loc[influencer, 'influenced_numbers'] += 1 / influencer_num
        # print('now:', artists.at[influencer, 'influenced_numbers'])

print(artists.head(20))


                         name      id decade       genre  \
0               The Exploited  759491   1980    Pop/Rock   
1                      Tricky   25462   1990  Electronic   
2                   Bob Dylan   66915   1960    Pop/Rock   
3               Leonard Cohen   71209   1950    Pop/Rock   
4                The Gun Club   91438   1980    Pop/Rock   
5              Chrissie Hynde  113823   1970    Pop/Rock   
6   Crime & the City Solution  132710   1980    Pop/Rock   
7                 Lesley Gore  208582   1960    Pop/Rock   
8              Massive Attack  378288   1980  Electronic   
9                   Nick Cave  397880   1970    Pop/Rock   
10              The Raincoats  489999   1970    Pop/Rock   
11                 Pretenders  492331   1970    Pop/Rock   
12                  Tom Waits  615119   1970    Pop/Rock   
13         Marianne Faithfull  651107   1960    Pop/Rock   
14                Patti Smith  747445   1970    Pop/Rock   
15                Sonic Youth  755156   

In [116]:
print(artists.sort_values('influenced_numbers', ascending= False)[['name', 'decade', 'influenced_numbers']].head(50))

                        name decade  influenced_numbers
64               The Beatles   1960           72.915701
2                  Bob Dylan   1960           39.992121
131       The Rolling Stones   1960           28.950855
481              David Bowie   1960           23.556406
673            Hank Williams   1930           23.041072
49              Led Zeppelin   1960           20.535912
109              Miles Davis   1940           19.533789
54              Jimi Hendrix   1960           18.587742
480                Sam Cooke   1950           18.545972
297           Charlie Parker   1930           18.196015
234            Elvis Presley   1950           18.003767
368              James Brown   1950           17.905301
182                The Kinks   1960           17.543028
98            The Beach Boys   1960           16.760319
193              Marvin Gaye   1950           16.687324
482                Brian Eno   1970           16.190051
65             Black Sabbath   1960           15

### 发现那个影响人数极多的人正是Bob Dylan

In [96]:
print(artists.loc[2, 'towards'])
print(artists.iloc[2])

[489, 678, 42, 2692, 2816, 1888, 648, 276, 462, 2440, 762, 1323, 3805, 695, 133, 839, 649, 1872, 1237, 1112, 2470, 3223, 605, 2823, 2199, 1292, 3192, 465, 97, 261, 1496, 1286, 946, 357, 1517, 834, 2223, 1368, 696, 382, 1300, 2767, 1486, 596, 3, 1070, 3949, 3954, 680, 466, 725, 902, 3973, 1487, 1970, 3249, 1824, 1800, 835, 2003, 2119, 606, 3232, 681, 2620, 1406, 650, 241, 884, 765, 424, 1407, 4073, 2459, 2903, 691, 1185, 282, 48, 661, 152, 113, 1582, 242, 2940, 608, 1248, 1238, 731, 4134, 703, 662, 4148, 885, 2786, 610, 574, 1293, 4163, 949, 2570, 1948, 1035, 905, 1054, 3534, 4202, 732, 1912, 3683, 613, 1055, 124, 1924, 941, 1769, 1503, 1884, 2794, 1045, 4249, 614, 799, 3042, 403, 3590, 1316, 1200, 1513, 1520, 849, 433, 1488, 733, 850, 3653, 2370, 640, 2438, 1429, 615, 2328, 425, 2651, 4364, 4368, 434, 1539, 836, 2217, 4381, 977, 2652, 2400, 4394, 616, 2412, 267, 4400, 1881, 192, 1116, 617, 734, 26, 2025, 2618, 134, 1504, 2745, 1162, 851, 471, 907, 472, 713, 473, 4433, 388, 1992, 1206, 

## 得到每个年代的艺术家人数

In [97]:
decade_artists = artists['decade'].value_counts()
decade_artists = decade_artists.sort_index()
print(decade_artists)
decade_genre_artists = artists[['decade', 'genre']].value_counts()
print(decade_genre_artists)

1930     155
1940     245
1950     540
1960     855
1970     796
1980     951
1990    1129
2000     728
2010     204
Name: decade, dtype: int64
decade  genre         
1990    Pop/Rock          665
1980    Pop/Rock          612
2000    Pop/Rock          480
1970    Pop/Rock          419
1960    Pop/Rock          410
                         ... 
1990    Easy Listening      1
2010    Classical           1
        Easy Listening      1
        Latin               1
1970    Easy Listening      1
Length: 144, dtype: int64


### 整合到artists里

In [104]:
artists['decade_artist'] = pd.Series([0] * artists.shape[0])
artists['decade_genre_artist'] = pd.Series([0] * artists.shape[0])
artists['next_decade_artist'] = pd.Series([0] * artists.shape[0])
artists['next_decade_genre_artist'] = pd.Series([0] * artist.shape[0])
for index, row in artists.iterrows():
    decade = row['decade']
    genre = row['genre']
    artists.loc[index, 'decade_artist'] = decade_artists[decade]
    artists.loc[index, 'decade_genre_artist'] = decade_genre_artists[(decade, genre)]
    next_decade = decade + 10
    if next_decade in decade_artists.index:
        artists.loc[index, 'next_decade_artist'] = decade_artists[next_decade]
    else:
        artists.loc[index, 'next_decade_artist'] = 0
    if (next_decade, genre) in decade_genre_artists.index:
        artists.loc[index, 'next_decade_genre_artist'] = decade_genre_artists[(next_decade, genre)]
    else:
        artists.loc[index, 'next_decade_genre_artist'] = 0
print(artists.head())

            name      id decade       genre  \
0  The Exploited  759491   1980    Pop/Rock   
1         Tricky   25462   1990  Electronic   
2      Bob Dylan   66915   1960    Pop/Rock   
3  Leonard Cohen   71209   1950    Pop/Rock   
4   The Gun Club   91438   1980    Pop/Rock   

                                             towards  \
0  [3774, 3919, 1382, 1383, 1544, 211, 4530, 1364...   
1  [489, 1626, 3130, 3545, 4169, 2966, 4367, 1539...   
2  [489, 678, 42, 2692, 2816, 1888, 648, 276, 462...   
3  [489, 3223, 840, 1889, 3949, 3682, 2657, 6, 24...   
4  [489, 947, 3177, 714, 1831, 2954, 17, 1750, 5161]   

                                                from  influenced_numbers  \
0  [680, 1171, 1359, 1315, 1838, 1599, 361, 1675,...            1.965546   
1  [777, 1005, 1006, 478, 1007, 1008, 228, 195, 1...            1.113818   
2  [1697, 1375, 1446, 1698, 1699, 233, 1700, 1447...           39.992121   
3  [2, 1486, 1757, 424, 1134, 1054, 480, 425, 673...            5.628317   


## 寻找Taylor Swift

In [117]:
idTS = id2art[472102]
print(artists.iloc[idTS])

name                                                             Taylor Swift
id                                                                     472102
decade                                                                   2000
genre                                                                Pop/Rock
towards                                                                [5424]
from                        [428, 595, 430, 431, 432, 78, 433, 2822, 3056,...
influenced_numbers                                                        0.2
decade_artist                                                             728
decade_genre_artist                                                       480
next_decade_artist                                                        204
next_decade_genre_artist                                                  116
Name: 3657, dtype: object


In [131]:
indexs = set()
def dfs(artist, gen):
    global indexs
    # print(indexs)
    if gen == -1:
        return;
    if artist in indexs:
        return;
    indexs.add(artist)
    # print(artists.at[artist, 'towards'])
    for next_artist in artists.at[artist, 'towards']:
        dfs(next_artist, gen - 1)
    for next_artist in artists.at[artist, 'from']:
        dfs(next_artist, gen - 1)

dfs(idTS, 4)


In [132]:
print(indexs)

{2, 3, 4102, 9, 13, 15, 2089, 2090, 2099, 55, 64, 67, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 2125, 2132, 4176, 93, 96, 97, 98, 2145, 101, 109, 112, 4237, 2189, 143, 2199, 2203, 166, 4263, 4272, 178, 180, 2235, 189, 194, 2279, 234, 2282, 4333, 248, 2301, 269, 4374, 4383, 288, 291, 292, 295, 300, 2354, 4407, 315, 342, 2404, 2405, 2406, 4456, 369, 377, 2426, 379, 388, 394, 2449, 2454, 2467, 428, 429, 430, 431, 432, 433, 434, 435, 437, 439, 441, 442, 2491, 4553, 490, 493, 495, 4592, 497, 502, 503, 504, 507, 2555, 2556, 523, 524, 2572, 2573, 527, 528, 2583, 544, 545, 546, 549, 4661, 2625, 2626, 579, 2628, 2638, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 2653, 2652, 606, 611, 613, 614, 616, 617, 4735, 640, 2695, 2697, 650, 670, 671, 672, 673, 674, 675, 676, 677, 2749, 701, 703, 704, 2760, 2761, 4819, 2772, 2774, 729, 735, 736, 743, 4845, 2808, 4857, 765, 2814, 770, 2821, 2822, 775, 776, 782, 2840, 816, 820, 826, 827, 828, 829, 830, 4928, 832, 833, 854, 4955, 860, 2912

## 输出给CC

In [133]:
# artists_print = artists.drop(['towards', 'from'], axis= 1)
artists_print = artists.loc[indexs]
print(artists_print)
artists_print.to_csv('output/forCC.csv', index= False)

                        name      id decade     genre  \
2                  Bob Dylan   66915   1960  Pop/Rock   
3              Leonard Cohen   71209   1950  Pop/Rock   
4102              BarlowGirl  139212   2000  Pop/Rock   
9                  Nick Cave  397880   1970  Pop/Rock   
13        Marianne Faithfull  651107   1960  Pop/Rock   
...                      ...     ...    ...       ...   
1952      Christina Aguilera  780258   1990  Pop/Rock   
1963  Mother Maybelle Carter  337856   1950   Country   
1964           Hank Thompson  947816   1940   Country   
1970             Kevin Coyne   83859   1970  Pop/Rock   
2047            Bobby Vinton  766066   1950     Vocal   

                                                towards  \
2     [489, 678, 42, 2692, 2816, 1888, 648, 276, 462...   
3     [489, 3223, 840, 1889, 3949, 3682, 2657, 6, 24...   
4102                                                 []   
9     [489, 3012, 1390, 765, 3664, 2442, 1036, 2925,...   
13    [489, 2620, 10